In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperSphere, HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [56]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
# kt=1
km = kt*thrustToTorque #4.4733e-8

# h = 1/500
h = 1/20

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [57]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [58]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [59]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)

    tol = 1e-4
    x_axis = [1.0; 0.0; 0.0]

    # Compute axis angle
    dot = x_axis'*dir
    if dot > 1-tol
        axis = x_axis
    elseif dot < -1+tol
        axis = -x_axis
    else
        axis = cross(x_axis, dir)
    end
    angle = acos(x_axis'*dir)

    settransform!(visObject, Translation(center...) ∘ LinearMap(AngleAxis(angle, axis...)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [7]:
# bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
# setobject!(vis["testsphere"], HyperSphere(Point(1.0, 0.0, 0.5), 0.2))
# setobject!(vis["testsphere_avoid"], HyperSphere(Point(1.0, 0.0, 0.5), 0.5), bounding_material)
# delete!(vis["testsphere"])
# delete!(vis["testsphere_avoid"])

In [112]:
Tfinal = 15.0
N = Int(Tfinal/h)+1

# Cost weights
max_dev_x = [0.1; 0.1; 0.1; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.5; 0.5; 0.5; 0.5]
Q = spdiagm(1 ./(max_dev_x.^2))
R = spdiagm(1 ./(max_dev_u.^2))
Qf = Q

# # Number of LQR direct solve matrix sets to cache
ρ_list_length = 4

# Allocate cache
cache = (
    A = Ãdyn,
    B = B̃dyn,

    # ρ = diagm(ones(nu)),
    # Kinf = zeros(nu,nx),
    # Pinf = zeros(nx,nx),
    # Quu_inv = zeros(nu,nu),
    # AmBKt = zeros(nx,nx), 
    # coeff_d2p = zeros(nx,nu), 

    ρ_list = [diagm(ones(nu)) for _ in 1:ρ_list_length],
    Kinf_list = [zeros(nu,nx) for _ in 1:ρ_list_length],
    Pinf_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    Quu_inv_list = [zeros(nu,nu) for _ in 1:ρ_list_length],
    AmBKt_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    coeff_d2p_list = [zeros(nx,nu) for _ in 1:ρ_list_length],
)

# Precompute all cached matrices for multiple ρ values starting at ρ = .1 and multiplying by 5
for k = 1:ρ_list_length
    # ρ
    ρ = diagm([1; 1; 1; 1])*.1*5^(k-1)
    # ρ = diagm([5; 5; 5; 5])
    # ρ = 5
    R̃ = R + ρ*I;
    # Compute Kinf, Pinf
    Nriccati = 10000
    P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
    K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain
    P[Nriccati] .= Qf;
    for k = (N-1):-1:1
        K[k] = (R̃ + cache.B'*P[k+1]*cache.B)\(cache.B'*P[k+1]*cache.A);
        P[k] = Q + cache.A'*P[k+1]*(cache.A - cache.B*K[k]);
    end

    # Cache precomputed values
    cache.ρ_list[k] .= ρ;
    cache.Kinf_list[k] .= K[1];
    cache.Pinf_list[k] .= P[1];
    cache.Quu_inv_list[k] .= (R̃ + cache.B'*cache.Pinf_list[k]*cache.B)\I;
    cache.AmBKt_list[k] .= (cache.A - cache.B*cache.Kinf_list[k])';
    cache.coeff_d2p_list[k] .= cache.Kinf_list[k]'*R̃ - cache.AmBKt_list[k]*cache.Pinf_list[k]*cache.B;
end

# Cache precomputed values
# cache.ρ .= ρ;
# cache.Kinf .= K[1];
# cache.Pinf .= P[1];
# cache.Quu_inv .= (R̃ + cache.B'*cache.Pinf*cache.B)\I;
# cache.AmBKt .= (cache.A - cache.B*cache.Kinf)';
# cache.coeff_d2p .= cache.Kinf'*R̃ - cache.AmBKt*cache.Pinf*cache.B;


# Create reference sketch trajectory
X̃ref = [zeros(nx) for i = 1:N]
Xref = [[0; 0; .5; 1; zeros(9)] for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [9]:
# display(cache.Kinf)
# display(cache.Pinf)
# display(cache.Quu_inv)
# display(cache.AmBKt)
# display(cache.coeff_d2p)

cache_size = 4*( *(size(cache.Kinf)...) + *(size(cache.Pinf)...) + *(size(cache.Quu_inv)...) + *(size(cache.AmBKt)...) + *(size(cache.coeff_d2p)...) )

ErrorException: type NamedTuple has no field Kinf

In [121]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))



# Clean up MeshCat environment from previous run
for k in 1:10000
    delete!(vis["xlin " * string(k)])
    delete!(vis["linearized constraint " * string(k)])
    delete!(vis["obstacle " * string(k)])
    delete!(vis["bounding obstacle " * string(k)])
end



# Used for skipping skip_obs number of linearized obstacle constraints (skips both visualization and computation)
skip_obs = 1

# MPC setup
Nh = 20


# Create obstacle in MeshCat
obstacle_center = [0.0; 2.0; 0.5]
obstacle_velocity = 1 * h

r_vis = 0.1
r_obstacle = 0.75

# Create visualization data to populate during MPC solution

bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for k in 1:skip_obs:Nh]


# Instantiate TinyMPC-ADMM variables

x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
# xmin = [-Inf .* ones(nx) for i = 1:Nh]
# xmax = [[Inf; 0.5; Inf .* ones(10)] for i = 1:Nh]
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
xmax = [zeros(1) for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]

for k = 1:skip_obs:Nh
    xc = obstacle_center - X̃ref[k][1:3]
    a = xc/norm(xc)

    Aineq[k][1:3] = a'

    q_c = obstacle_center - r_obstacle*a
    b = a'*q_c
    xmax[k][1] = b
    TransformPlane(constraintPlanes[cld(k,skip_obs)], q_c, a)
end



# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]
@time for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # State slack (auxiliary) and dual variables
    v = [zeros(nx) for i = 1:Nh]
    vnew = [zeros(nx) for i = 1:Nh]
    g = [zeros(nx) for i = 1:Nh]

    # Input slack (auxiliary) and dual variables
    z = [zeros(nu) for i = 1:Nh-1]
    znew = [zeros(nu) for i = 1:Nh-1]
    y = [zeros(nu) for i = 1:Nh-1]

    x[1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]

    # params.constraint_A .= Aineq
    # params.umin .= umin
    # params.umax .= umax
    # params.xmin .= xmin
    # params.xmax .= xmax
    # params.Xref .= X̃ref[k:k+Nh-1]
    # params.Uref .= Uref[k:k+Nh-2]
    
    params = (
        N = Nh,
        Q = Q, R = R, Qf = Qf,
        constraint_A = Aineq,
        umin = umin, umax = umax,
        xmin = xmin, xmax = xmax,
        Xref = X̃ref[k:k+Nh-1], Uref = Uref[k:k+Nh-2], 
        cache = cache,

        ρ_index = [1]
    )

    # Solve
    Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=200)
    # @show iter, status
    display("iter: " * string(iter))
    
    # Roll out x_k+1 with actual dynamics
    uhist[k] = Δu[1] + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
    
    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh]
    x_real[1] = xhist[k]
    for j = 1:Nh-1
        x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    end

    # Visualize solution x
    x_linearized = [Point(xp[1], xp[2], xp[3]+0.01) for xp in x_real]
    if k > 1
        x_lin_name_prev = "xlin " * string(k-1)
        setvisible!(vis[x_lin_name_prev], false)
    end
    x_lin_name = "xlin " * string(k)
    setobject!(vis[x_lin_name], Object(PointCloud(x_linearized), 
        LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Create constraints for each knot point in the horizon
    #   assuming the obstacle's center moves at constant velocity between each step
    obstacle_to_quad = x_real[1][1:3] - obstacle_center
    obstacle_to_quad_dir = obstacle_to_quad / norm(obstacle_to_quad)
    obstacle_dist_traveled = obstacle_velocity * obstacle_to_quad_dir
    
    # Update linearized obstacle constraints
    if k > 1
        for i = 1:Nh
            setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
        end
    end
    constraintPlanes = [setobject!(vis["linearized constraint " * string((k-1)*Nh + i)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for i in 1:skip_obs:Nh]
    for j = 1:skip_obs:Nh
        obstacle_center_prediction = obstacle_center + obstacle_dist_traveled*j
        xc = obstacle_center_prediction - x_real[j][1:3]
        a = xc/norm(xc) # hyperplane projection expects normalized a

        Aineq[j][1:3] = a'

        q_c = obstacle_center_prediction - r_obstacle*a
        b = a'*q_c
        xmax[j][1] = b
        TransformPlane(constraintPlanes[cld(j,skip_obs)], q_c, a)
    end

    # Update actual obstacle location at x=0
    obstacle_center += obstacle_dist_traveled
    
    # # Visualize cylinder
    # if k > 1
    #     setvisible!(vis["obstacle " * string(k-1)], false)
    #     setvisible!(vis["bounding obstacle " * string(k-1)], false)
    # end
    # cyl_vis = setobject!(vis["obstacle " * string(k)], Cylinder(Point(0, 0, -0.5), Point(0, 0, 0.5), r_vis))
    # cyl_avoid = setobject!(vis["bounding obstacle " * string(k)], Cylinder(Point(0, 0, -5.05), Point(0, 0, 5.05), r_obstacle), bounding_material)
    # settransform!(cyl_vis, Translation(obstacle_center[1], obstacle_center[2], obstacle_center[3]))

    # Visualize sphere
    if k > 1
        setvisible!(vis["obstacle " * string(k-1)], false)
        setvisible!(vis["bounding obstacle " * string(k-1)], false)
    end
    setobject!(vis["obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_vis))
    setobject!(vis["bounding obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_obstacle), bounding_material)
end

# delete!(vis["xHistline"])
# xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
# setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
# LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

"iter: 3"

"iter: 1"

"iter: 1"

"iter: 1"

"iter: 1"

...

"updating ρ from 0.1 to 2.5"

"iter: 191"

"updating ρ from 0.1 to 2.5"

"iter: 201"

 17.680384 seconds (43.10 M allocations: 6.181 GiB, 18.58% gc time, 10.80% compilation time)


In [10]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)

In [108]:
# constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:Nh]
for k in 1:N
    delete!(vis["linearized constraint " * string(k)])
end

# Create MeshCat animation
anim = MeshCat.Animation()

for k = 1:Nsim-1
    atframe(anim, k-1) do
        # Set visibility of each object to false
        if k == 1
            for j = 1:Nsim
                for i = 1:Nh
                    setvisible!(vis["linearized constraint " * string((j-1)*Nh + i)], false)
                end
                setvisible!(vis["xlin " * string(j)], false)
                setvisible!(vis["obstacle " * string(j)], false)
                setvisible!(vis["bounding obstacle " * string(j)], false)
            end
        end

        # Change visible obstacle
        setvisible!(vis["obstacle " * string(k)], true)
        setvisible!(vis["bounding obstacle " * string(k)], true)
        if k > 1
            setvisible!(vis["obstacle " * string(k-1)], false)
            setvisible!(vis["bounding obstacle " * string(k-1)], false)
        end


        # Update obstacle constraint visuals
        for i = 1:Nh
            setvisible!(vis["linearized constraint " * string((k-1)*Nh + i)], true)
        end
        if k > 1
            for i = 1:Nh
                setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
            end
        end
        

        # Change visible horizon
        setvisible!(vis["xlin " * string(k)], true)
        if k > 1
            setvisible!(vis["xlin " * string(k-1)], false)
        end

        # Update quadrotor transform
        xbar = RBState(quad_model, xhist[k])
        if quad_model.ned
            rq = position(xbar)
            vq = linear_velocity(xbar)
            rq = SA[rq[1],-rq[2],-rq[3]]
            vq = SA[vq[1],-vq[2],-vq[3]]
            xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
        end
        settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    end
end

setanimation!(vis, anim)

In [150]:
MeshCat.convert_frames_to_video(
    "/home/sam/Git/tinympc-julia/state_constraints/state_constraints_tinympc.tar")

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 8.1.0 (GCC)
  configuration: --enable-cross-compile --cross-prefix=/opt/x86_64-linux-gnu/bin/x86_64-linux-gnu- --arch=x86_64 --target-os=linux --cc=cc --cxx=c++ --dep-cc=cc --ar=ar --nm=nm --sysinclude=/workspace/destdir/include --pkg-config=/usr/bin/pkg-config --pkg-config-flags=--static --prefix=/workspace/destdir --sysroot=/opt/x86_64-linux-gnu/x86_64-linux-gnu/sys-root --extra-libs=-lpthread --enable-gpl --enable-version3 --enable-nonfree --disable-static --enable-shared --enable-pic --disable-debug --disable-doc --enable-avresample --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libx264 --enable-libx265 --enable-libvpx --enable-encoders --enable-decoders --enable-muxers --enable-demuxers --enable-parsers --enable-openssl --disable-schannel --extra-cflags=-I/workspace/destdir/include --extra-ldflags=-L/workspac

[libx264 @ 0x207aa80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x207aa80] profile High 4:4:4 Predictive, level 4.2, 4:4:4, 8-bit
[libx264 @ 0x207aa80] 264 - core 163 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=5 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=8 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=2 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=24 lookahead_threads=4 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=3 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=50 rc=crf mbtree=1 crf=18.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/sam/Git/tinympc-julia/state_constraints/output.mp4':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: h264 (avc1 

frame= 2359 fps= 93 q=-1.0 Lsize=    5154kB time=00:00:39.26 bitrate=1075.3kbits/s dup=1 drop=0 speed=1.55x    
video:5126kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.553680%
[libx264 @ 0x207aa80] frame I:10    Avg QP:13.23  size:266055
[libx264 @ 0x207aa80] frame P:605   Avg QP:19.08  size:  2482
[libx264 @ 0x207aa80] frame B:1744  Avg QP:24.76  size:   623
[libx264 @ 0x207aa80] consecutive B-frames:  0.5%  1.6%  3.4% 94.4%
[libx264 @ 0x207aa80] mb I  I16..4: 17.6% 19.8% 62.6%
[libx264 @ 0x207aa80] mb P  I16..4:  0.1%  0.1%  0.2%  P16..4:  2.0%  1.2%  0.7%  0.0%  0.0%    skip:95.7%
[libx264 @ 0x207aa80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  2.6%  0.4%  0.1%  direct: 0.1%  skip:96.8%  L0:55.4% L1:41.7% BI: 2.9%
[libx264 @ 0x207aa80] 8x8 transform intra:22.4% inter:29.5%
[libx264 @ 0x207aa80] direct mvs  spatial:99.3% temporal:0.7%
[libx264 @ 0x207aa80] coded y,u,v intra: 48.1% 41.2% 39.9% inter: 0.3% 0.2% 0.1%
[libx264 @ 0x207aa80] i16 v,h,dc,p:

┌ Info: Saved output as /home/sam/Git/tinympc-julia/state_constraints/output.mp4
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/animations.jl:121


"/home/sam/Git/tinympc-julia/state_constraints/output.mp4"